In [12]:
import requests
import pandas as pd

In [13]:
# Function to query the API and fetch the data
def fetch_data(offset=0, rows_per_request=100):
    base_url = "https://search.worldbank.org/api/v2/wds"
    params = {
        "qterm": "hepatitis",
        "format": "json",
        "os": offset,
        "rows": rows_per_request,
        "fl": "issn,txturl,abstracts,guid,docna,count,authr,colti,display_title,docdt,docty,geo_reg,id,isbn,keywd,lang,majtheme,pdfurl,subsc,subtopic,theme,topic,url"
    }
    
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [14]:
def process_data(data):
    documents = data.get('documents', {})
    
    flat_data = []
    for doc_id, doc in documents.items():
        # Initialize a dictionary for the flattened data
        flat_doc = {'doc_id': doc_id}
        for key, value in doc.items():
            # Check if the key is one of the special fields and if the value is a string
            if key in ['authors', 'geo_regions', 'keywords'] and isinstance(value, str):
                try:
                    # Attempt to parse the JSON string into a dictionary
                    value = json.loads(value)
                except json.JSONDecodeError:
                    # Handle cases where parsing fails
                    print(f"Error decoding JSON for {key} in document {doc_id}")
            
            if isinstance(value, dict):
                # Handle nested dictionaries
                nested_values = []
                for nested_key, nested_value in value.items():
                    if isinstance(nested_value, dict) and 'value' in nested_value:
                        nested_values.append(nested_value['value'])
                    elif isinstance(nested_value, str):
                        nested_values.append(nested_value)
                    else:
                        nested_values.append(str(nested_value))
                flat_doc[key] = '; '.join(nested_values)
            else:
                flat_doc[key] = value
        flat_data.append(flat_doc)
    return flat_data

In [15]:
# Main function to fetch all data and write to an Excel file
def main():
    all_data = []
    offset = 0
    total_rows = 802  # Assuming the total number of documents is known. This could also be dynamically retrieved.
    
    while offset < total_rows:
        data = fetch_data(offset=offset)
        if data:
            processed_data = process_data(data)
            all_data.extend(processed_data)
            offset += 100  # Increment the offset for the next loop iteration
        else:
            print("Failed to fetch data.")
            break
    
    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(all_data)
    
    # Write the DataFrame to an Excel file
    excel_path = "/Users/willit/Documents/WorldBank/output2.xlsx"
    df.to_excel(excel_path, index=False)
    print(f"Data successfully written to {excel_path}")


In [16]:
if __name__ == "__main__":
    main()


Data successfully written to /Users/willit/Documents/WorldBank/output2.xlsx
